## 1️⃣ Setup: Mount Google Drive & Install Packages

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Install required packages
!pip install open_clip_torch pytorch-lightning albumentations grad-cam seaborn -q

## 2️⃣ Upload Project Files

**Şu dosyaları Google Drive'a yükleyin:**
1. `FetalCLIP_config.json`
2. `FetalCLIP_weights.pt`
3. `preprocessed_data/` klasörünü zip'leyin → `preprocessed_data.zip`

**Drive'da şu yapıyı oluşturun:**
```
My Drive/
  FetalBrain/
    FetalCLIP_config.json
    FetalCLIP_weights.pt
    preprocessed_data.zip
```

In [ ]:
# Extract preprocessed data
import zipfile
import os

# Drive paths (değiştirin gerekirse)
DRIVE_DIR = '/content/drive/MyDrive/FetalBrain'
WORK_DIR = '/content/fetal_brain'

os.makedirs(WORK_DIR, exist_ok=True)

# Extract preprocessed data
print("Extracting preprocessed_data.zip...")
with zipfile.ZipFile(f'{DRIVE_DIR}/preprocessed_data.zip', 'r') as zip_ref:
    zip_ref.extractall(WORK_DIR)

print("✓ Data extracted!")
!ls -lh {WORK_DIR}

## 3️⃣ Training Script

In [ ]:
import os
import json
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm
import open_clip
from datetime import datetime
from sklearn.metrics import classification_report, confusion_matrix, f1_score
import matplotlib.pyplot as plt
import seaborn as sns

# Paths
PATH_FETALCLIP_CONFIG = f'{DRIVE_DIR}/FetalCLIP_config.json'
PATH_FETALCLIP_WEIGHT = f'{DRIVE_DIR}/FetalCLIP_weights.pt'
PREPROCESSED_DIR = f'{WORK_DIR}/preprocessed_data'
OUTPUT_DIR = f'{WORK_DIR}/results'

os.makedirs(OUTPUT_DIR, exist_ok=True)

# Config - BINARY CLASSIFICATION
CLASS_NAMES = ['Trans-thalamic', 'Other']
NUM_CLASSES = 2
DICT_CLSNAME_TO_CLSINDEX = {name: idx for idx, name in enumerate(CLASS_NAMES)}

BATCH_SIZE = 32  # Colab GPU için optimize
EPOCHS = 50
LEARNING_RATE = 0.001
DENSE_UNITS = 512
DROPOUT_RATE = 0.5

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🚀 Using device: {device}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

print(f"\n🎯 Binary Plane Classification (Gatekeeper Model)")
print(f"   - Trans-thalamic (0): Valid for HC measurement")
print(f"   - Other (1): Invalid planes")

In [ ]:
# Dataset
class FetalBrainDataset(Dataset):
    def __init__(self, preprocessed_dir, split='train', transform=None):
        self.preprocessed_dir = os.path.join(preprocessed_dir, split)
        self.transform = transform
        self.data = []
        
        for filename in os.listdir(self.preprocessed_dir):
            if not filename.endswith('.png'):
                continue
            
            class_name = None
            for cn in CLASS_NAMES:
                if cn in filename:
                    class_name = cn
                    break
            
            if class_name is not None:
                self.data.append({
                    'path': os.path.join(self.preprocessed_dir, filename),
                    'label': DICT_CLSNAME_TO_CLSINDEX[class_name],
                    'class_name': class_name
                })
        
        print(f"  {split.upper()}: {len(self.data)} images")
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        item = self.data[idx]
        image = Image.open(item['path']).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        return image, item['label']

In [ ]:
# Load FetalCLIP
print("\n📦 Loading FetalCLIP...")
with open(PATH_FETALCLIP_CONFIG, "r") as file:
    config_fetalclip = json.load(file)
open_clip.factory._MODEL_CONFIGS["FetalCLIP"] = config_fetalclip

model_fetalclip, _, preprocess = open_clip.create_model_and_transforms(
    "FetalCLIP", 
    pretrained=PATH_FETALCLIP_WEIGHT
)
model_fetalclip = model_fetalclip.to(device)
model_fetalclip.eval()

# Freeze FetalCLIP
for param in model_fetalclip.parameters():
    param.requires_grad = False

print("✓ FetalCLIP loaded and frozen")

In [ ]:
# Custom Classifier - BINARY OUTPUT
class FetalBrainClassifier(nn.Module):
    def __init__(self, embedding_dim=768, dense_units=512, num_classes=2, dropout_rate=0.5):
        super(FetalBrainClassifier, self).__init__()
        self.fc1 = nn.Linear(embedding_dim, dense_units)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout_rate)
        self.fc2 = nn.Linear(dense_units, num_classes)
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

classifier = FetalBrainClassifier(
    embedding_dim=768,
    dense_units=DENSE_UNITS,
    num_classes=NUM_CLASSES,
    dropout_rate=DROPOUT_RATE
).to(device)

print(f"\n🧠 Binary Classifier: {sum(p.numel() for p in classifier.parameters()):,} parameters")
print(f"   Architecture: FetalCLIP → Linear(768→512) → ReLU → Dropout(0.5) → Linear(512→2)")

In [ ]:
# Data Loaders
print("\n📊 Loading datasets...")
train_dataset = FetalBrainDataset(PREPROCESSED_DIR, 'train', preprocess)
val_dataset = FetalBrainDataset(PREPROCESSED_DIR, 'val', preprocess)
test_dataset = FetalBrainDataset(PREPROCESSED_DIR, 'test', preprocess)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)

print(f"\n✓ Batches: Train={len(train_loader)}, Val={len(val_loader)}, Test={len(test_loader)}")

In [ ]:
# Training setup
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(classifier.parameters(), lr=LEARNING_RATE)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=True)

# Training history
history = {
    'train_loss': [], 'train_acc': [], 'train_f1': [],
    'val_loss': [], 'val_acc': [], 'val_f1': []
}

best_val_f1 = 0.0
patience_counter = 0
EARLY_STOP_PATIENCE = 15

In [ ]:
# Training loop
print("\n" + "="*70)
print("🚀 STARTING TRAINING")
print("="*70)

for epoch in range(EPOCHS):
    print(f"\n📍 Epoch {epoch+1}/{EPOCHS}")
    
    # TRAIN
    classifier.train()
    train_loss = 0.0
    train_preds = []
    train_labels = []
    
    for images, labels in tqdm(train_loader, desc="Training"):
        images = images.to(device)
        labels = labels.to(device)
        
        # Extract embeddings
        with torch.no_grad():
            embeddings = model_fetalclip.encode_image(images)
            embeddings = embeddings / embeddings.norm(dim=-1, keepdim=True)
        
        # Forward
        outputs = classifier(embeddings)
        loss = criterion(outputs, labels)
        
        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        train_preds.extend(preds.cpu().numpy())
        train_labels.extend(labels.cpu().numpy())
    
    train_loss /= len(train_loader)
    train_acc = np.mean(np.array(train_preds) == np.array(train_labels))
    train_f1 = f1_score(train_labels, train_preds, average='weighted')
    
    # VALIDATION
    classifier.eval()
    val_loss = 0.0
    val_preds = []
    val_labels = []
    
    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc="Validation"):
            images = images.to(device)
            labels = labels.to(device)
            
            embeddings = model_fetalclip.encode_image(images)
            embeddings = embeddings / embeddings.norm(dim=-1, keepdim=True)
            
            outputs = classifier(embeddings)
            loss = criterion(outputs, labels)
            
            val_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            val_preds.extend(preds.cpu().numpy())
            val_labels.extend(labels.cpu().numpy())
    
    val_loss /= len(val_loader)
    val_acc = np.mean(np.array(val_preds) == np.array(val_labels))
    val_f1 = f1_score(val_labels, val_preds, average='weighted')
    
    # Update history
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['train_f1'].append(train_f1)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)
    history['val_f1'].append(val_f1)
    
    # Learning rate scheduler
    scheduler.step(val_loss)
    
    # Print metrics
    print(f"  Train - Loss: {train_loss:.4f}, Acc: {train_acc:.4f}, F1: {train_f1:.4f}")
    print(f"  Val   - Loss: {val_loss:.4f}, Acc: {val_acc:.4f}, F1: {val_f1:.4f}")
    
    # Save best model
    if val_f1 > best_val_f1:
        best_val_f1 = val_f1
        torch.save(classifier.state_dict(), f'{OUTPUT_DIR}/best_classifier.pt')
        print(f"  ✨ New best model saved! (F1: {best_val_f1:.4f})")
        patience_counter = 0
    else:
        patience_counter += 1
        print(f"  ⏳ Patience: {patience_counter}/{EARLY_STOP_PATIENCE}")
    
    # Early stopping
    if patience_counter >= EARLY_STOP_PATIENCE:
        print(f"\n⚠️ Early stopping triggered at epoch {epoch+1}")
        break

print("\n✅ Training completed!")

## 4️⃣ Evaluation & Results

In [ ]:
# Load best model
classifier.load_state_dict(torch.load(f'{OUTPUT_DIR}/best_classifier.pt'))
classifier.eval()

# Test evaluation
test_preds = []
test_labels = []

with torch.no_grad():
    for images, labels in tqdm(test_loader, desc="Testing"):
        images = images.to(device)
        
        embeddings = model_fetalclip.encode_image(images)
        embeddings = embeddings / embeddings.norm(dim=-1, keepdim=True)
        
        outputs = classifier(embeddings)
        _, preds = torch.max(outputs, 1)
        
        test_preds.extend(preds.cpu().numpy())
        test_labels.extend(labels.cpu().numpy())

# Metrics
test_acc = np.mean(np.array(test_preds) == np.array(test_labels))
test_f1 = f1_score(test_labels, test_preds, average='weighted')

print(f"\n{'='*70}")
print("📊 TEST RESULTS")
print(f"{'='*70}")
print(f"Accuracy: {test_acc:.4f}")
print(f"F1-Score: {test_f1:.4f}")
print(f"{'='*70}")

# Classification report
print("\nClassification Report:")
print(classification_report(test_labels, test_preds, target_names=CLASS_NAMES))

# Save report
with open(f'{OUTPUT_DIR}/classification_report.txt', 'w') as f:
    f.write(classification_report(test_labels, test_preds, target_names=CLASS_NAMES))

In [ ]:
# Confusion Matrix
cm = confusion_matrix(test_labels, test_preds)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=CLASS_NAMES, yticklabels=CLASS_NAMES)
plt.title('Confusion Matrix - Test Set', fontsize=16, fontweight='bold')
plt.ylabel('True Label', fontsize=12)
plt.xlabel('Predicted Label', fontsize=12)
plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/confusion_matrix.png', dpi=300)
plt.show()

print(f"\n✓ Confusion matrix saved to: {OUTPUT_DIR}/confusion_matrix.png")

In [ ]:
# Training curves
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Loss
axes[0].plot(history['train_loss'], label='Train', linewidth=2)
axes[0].plot(history['val_loss'], label='Validation', linewidth=2)
axes[0].set_xlabel('Epoch', fontsize=12)
axes[0].set_ylabel('Loss', fontsize=12)
axes[0].set_title('Training & Validation Loss', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Accuracy
axes[1].plot(history['train_acc'], label='Train', linewidth=2)
axes[1].plot(history['val_acc'], label='Validation', linewidth=2)
axes[1].set_xlabel('Epoch', fontsize=12)
axes[1].set_ylabel('Accuracy', fontsize=12)
axes[1].set_title('Training & Validation Accuracy', fontsize=14, fontweight='bold')
axes[1].legend()
axes[1].grid(alpha=0.3)

# F1-Score
axes[2].plot(history['train_f1'], label='Train', linewidth=2)
axes[2].plot(history['val_f1'], label='Validation', linewidth=2)
axes[2].set_xlabel('Epoch', fontsize=12)
axes[2].set_ylabel('F1-Score', fontsize=12)
axes[2].set_title('Training & Validation F1-Score', fontsize=14, fontweight='bold')
axes[2].legend()
axes[2].grid(alpha=0.3)

plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/training_curves.png', dpi=300)
plt.show()

print(f"\n✓ Training curves saved to: {OUTPUT_DIR}/training_curves.png")

In [ ]:
# Save results to Drive
import shutil

print("\n💾 Copying results to Google Drive...")
drive_results = f'{DRIVE_DIR}/results'
os.makedirs(drive_results, exist_ok=True)

# Copy files
shutil.copy(f'{OUTPUT_DIR}/best_classifier.pt', drive_results)
shutil.copy(f'{OUTPUT_DIR}/classification_report.txt', drive_results)
shutil.copy(f'{OUTPUT_DIR}/confusion_matrix.png', drive_results)
shutil.copy(f'{OUTPUT_DIR}/training_curves.png', drive_results)

print(f"✅ All results saved to: {drive_results}")
print("\n🎉 Training completed successfully!")